# 🔄 RPA Kestra - Extração de Status de Execução

Este notebook implementa um RPA (Robotic Process Automation) para extrair automaticamente o status de execução dos flows do Kestra.

## 📋 Objetivo
- Extrair dados de status dos flows do namespace `rpa.varejofacil`
- Monitorar execuções e seus status
- Gerar relatórios automatizados

## 🔗 URL Base
https://kestra.vissimo.tech/ui/main/flows?namespace=rpa.varejofacil&scope=USER&size=100&page=1&filters[scope][EQUALS]=USER&filters[namespace][PREFIX]=rpa.varejofacil


## 📦 Instalação e Importação de Bibliotecas


In [3]:
# Instalar bibliotecas necessárias (execute apenas se não estiverem instaladas)
# !pip install requests beautifulsoup4 selenium pandas matplotlib seaborn plotly

import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import time
import warnings
warnings.filterwarnings('ignore')

# Para automação web (se necessário)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# Para parsing HTML
from bs4 import BeautifulSoup

print("✅ Bibliotecas importadas com sucesso!")


✅ Bibliotecas importadas com sucesso!


## 🔧 Configurações e Credenciais


In [4]:
# Configurações da API Kestra
KESTRA_CONFIG = {
    'base_url': 'https://api.evino.com.br/kestra',
    'api_key': 'ELFkM8LTsLpTmbvzVyVqCslVDfBdNACP',
    'tenant': 'main',
    'namespace': 'rpa.varejofacil'
}

# URL da interface web
WEB_URL = "https://kestra.vissimo.tech/ui/main/flows?namespace=rpa.varejofacil&scope=USER&size=100&page=1&filters[scope][EQUALS]=USER&filters[namespace][PREFIX]=rpa.varejofacil"

# Headers para requisições
HEADERS = {
    'X-EVINO-KESTRA-API-KEY': KESTRA_CONFIG['api_key'],
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'User-Agent': 'MonitorDW/1.0'
}

print("🔧 Configurações carregadas:")
print(f"   Base URL: {KESTRA_CONFIG['base_url']}")
print(f"   Tenant: {KESTRA_CONFIG['tenant']}")
print(f"   Namespace: {KESTRA_CONFIG['namespace']}")
print(f"   Web URL: {WEB_URL}")


🔧 Configurações carregadas:
   Base URL: https://api.evino.com.br/kestra
   Tenant: main
   Namespace: rpa.varejofacil
   Web URL: https://kestra.vissimo.tech/ui/main/flows?namespace=rpa.varejofacil&scope=USER&size=100&page=1&filters[scope][EQUALS]=USER&filters[namespace][PREFIX]=rpa.varejofacil


## 🌐 Método 1: API REST (Recomendado)


In [5]:
def test_kestra_api_connection():
    """Testa a conexão com a API do Kestra"""
    try:
        # Teste básico de conectividade
        url = f"{KESTRA_CONFIG['base_url']}/api/v1/{KESTRA_CONFIG['tenant']}/namespaces"
        response = requests.get(url, headers=HEADERS, timeout=10)
        
        print(f"🔍 Testando conexão com: {url}")
        print(f"📊 Status Code: {response.status_code}")
        
        if response.status_code == 200:
            print("✅ Conexão bem-sucedida!")
            return True, response.json()
        elif response.status_code == 403:
            print("❌ Acesso negado (403) - Verificar permissões da API Key")
            return False, response.text
        elif response.status_code == 401:
            print("❌ Não autorizado (401) - Verificar API Key")
            return False, response.text
        else:
            print(f"❌ Erro inesperado: {response.status_code}")
            return False, response.text
            
    except requests.exceptions.Timeout:
        print("❌ Timeout na conexão")
        return False, "Timeout"
    except requests.exceptions.ConnectionError:
        print("❌ Erro de conexão")
        return False, "Connection Error"
    except Exception as e:
        print(f"❌ Erro inesperado: {str(e)}")
        return False, str(e)

# Testar conexão
api_connected, api_response = test_kestra_api_connection()
print(f"\n📋 Resposta da API: {api_response}")


🔍 Testando conexão com: https://api.evino.com.br/kestra/api/v1/main/namespaces
📊 Status Code: 403
❌ Acesso negado (403) - Verificar permissões da API Key

📋 Resposta da API: {"message":"Invalid authentication credentials"}


In [6]:
def get_kestra_flows():
    """Obtém lista de flows do Kestra"""
    try:
        url = f"{KESTRA_CONFIG['base_url']}/api/v1/{KESTRA_CONFIG['tenant']}/flows"
        response = requests.get(url, headers=HEADERS, timeout=10)
        
        if response.status_code == 200:
            flows = response.json()
            print(f"✅ {len(flows)} flows encontrados")
            return flows
        else:
            print(f"❌ Erro ao obter flows: {response.status_code}")
            return []
    except Exception as e:
        print(f"❌ Erro: {str(e)}")
        return []

def get_kestra_executions():
    """Obtém execuções do Kestra"""
    try:
        url = f"{KESTRA_CONFIG['base_url']}/api/v1/{KESTRA_CONFIG['tenant']}/executions"
        response = requests.get(url, headers=HEADERS, timeout=10)
        
        if response.status_code == 200:
            executions = response.json()
            print(f"✅ {len(executions)} execuções encontradas")
            return executions
        else:
            print(f"❌ Erro ao obter execuções: {response.status_code}")
            return []
    except Exception as e:
        print(f"❌ Erro: {str(e)}")
        return []

# Tentar obter dados via API
if api_connected:
    flows = get_kestra_flows()
    executions = get_kestra_executions()
else:
    print("⚠️ API não disponível, usando método alternativo (Web Scraping)")
    flows = []
    executions = []


⚠️ API não disponível, usando método alternativo (Web Scraping)


## 🕷️ Método 2: Web Scraping (Fallback)


In [7]:
def setup_selenium_driver():
    """Configura o driver do Selenium"""
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Executar sem interface gráfica
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--window-size=1920,1080')
    
    try:
        driver = webdriver.Chrome(options=chrome_options)
        return driver
    except Exception as e:
        print(f"❌ Erro ao configurar Selenium: {str(e)}")
        print("💡 Instale o ChromeDriver: https://chromedriver.chromium.org/")
        return None

def scrape_kestra_data():
    """Extrai dados do Kestra via web scraping"""
    driver = setup_selenium_driver()
    if not driver:
        return []
    
    try:
        print(f"🌐 Acessando: {WEB_URL}")
        driver.get(WEB_URL)
        
        # Aguardar carregamento da página
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "table"))
        )
        
        # Extrair dados da tabela
        flows_data = []
        
        # Encontrar todas as linhas da tabela
        rows = driver.find_elements(By.CSS_SELECTOR, "tbody tr")
        
        for row in rows:
            try:
                cells = row.find_elements(By.TAG_NAME, "td")
                if len(cells) >= 5:  # Verificar se tem colunas suficientes
                    flow_id = cells[0].text.strip()
                    namespace = cells[2].text.strip()
                    last_execution_date = cells[3].text.strip()
                    last_execution_status = cells[4].text.strip()
                    
                    flows_data.append({
                        'flow_id': flow_id,
                        'namespace': namespace,
                        'last_execution_date': last_execution_date,
                        'last_execution_status': last_execution_status,
                        'extracted_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    })
            except Exception as e:
                print(f"⚠️ Erro ao processar linha: {str(e)}")
                continue
        
        print(f"✅ {len(flows_data)} flows extraídos via web scraping")
        return flows_data
        
    except TimeoutException:
        print("❌ Timeout ao carregar página")
        return []
    except Exception as e:
        print(f"❌ Erro no web scraping: {str(e)}")
        return []
    finally:
        driver.quit()

# Executar web scraping se API não estiver disponível
if not api_connected or not flows:
    print("🕷️ Executando web scraping...")
    scraped_data = scrape_kestra_data()
else:
    scraped_data = []


🕷️ Executando web scraping...
🌐 Acessando: https://kestra.vissimo.tech/ui/main/flows?namespace=rpa.varejofacil&scope=USER&size=100&page=1&filters[scope][EQUALS]=USER&filters[namespace][PREFIX]=rpa.varejofacil
❌ Timeout ao carregar página
